# ES Theo Value vs Market Spread Analysis Pipeline (Polars + Plotly)

## Author:
Haoshu Wang (2025)

This script analyzes the relationship between observed S&P 500 E-mini Futures (ES) trade prices and their theoretical fair values implied by spot index levels, interest rates, and expected dividend points. It visualizes the spreads relative to spot prices and compares them against BTIC (Basis Trade at Index Close).

## Note: Currently the Theo_df is not ready yet, this notebook contains a section to generate a randomized Theo_df. Once the dataframe is ready, need to replace that!!!

## Main Components

### 1. Data Download & Load
- Downloads ES trade and BTIC quote data from Google Drive using `gdown`.
- Parses and loads both datasets using `polars` with efficient timestamp handling and type conversion.

### 2. Timestamp Unification and Forward-Filling
- Combines all unique timestamps from ES and BTIC.
- Performs backward `asof_join` and forward-fill to align the ES and BTIC data on a common timeline.
- Ensures no missing values before joining with theoretical prices.

### 3. Theoretical Futures Price Generation *(for visualization only)*
- Synthesizes a time series of spot index levels (`REALTIME_INDEXPOINTS`), expected dividend points, interest rates, and time-to-maturity (TTM).
- Computes theoretical fair value futures prices using the formula:

$$
\text{THEORETICAL_FUT} = \text{REALTIME_INDEXPOINTS} \cdot e^{r \cdot \text{TTM}} - \text{EXPECTED_DIVPOINTS}
$$

## Note: This is a placeholder and should be replaced with real index data for production use.

### 4. Spread Computation
- Merges the forward-filled ES and BTIC data with the theoretical futures data using `asof_join`.
- Computes the following spreads:
  - `ES_MINUS_SPOT = ES_TRADE - REALTIME_INDEXPOINTS`
  - `THEO_MINUS_SPOT = THEORETICAL_FUT - REALTIME_INDEXPOINTS`

### 5. Interactive Visualization
- Uses Plotly to generate an interactive time-series chart with three lines:
  - **ES - Spot** spread (solid)
  - **Theoretical - Spot** spread (solid)
  - **BTIC** value (dotted)
- Hover-enabled for detailed inspection of spread dynamics.

In [1]:
import gdown
import polars as pl
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [2]:
# Helper function to load trade and btic data
def load_trade_and_btic_data(trade_url: str, btic_url: str):
    gdown.download(trade_url, "trade_data.csv", quiet=False)
    gdown.download(btic_url, "btic_data.csv", quiet=False)

    trade_df = pl.read_csv("trade_data.csv", infer_schema_length=1000).select([
        pl.col("Date-Time").str.to_datetime(strict=False).alias("TIMESTAMP"),
        pl.col("Last").cast(pl.Float64).alias("ES_TRADE")
    ]).with_columns([
        pl.col("TIMESTAMP").cast(pl.Datetime)
    ])

    btic_df = pl.read_csv("btic_data.csv", infer_schema_length=1000).select([
        pl.col("Date-Time").str.to_datetime(strict=False).alias("TIMESTAMP"),
        pl.col("Close Bid").cast(pl.Float64).alias("BTIC")
    ]).with_columns([
        pl.col("TIMESTAMP").cast(pl.Datetime)
    ])

    return trade_df, btic_df

In [3]:
# Helper function to merge the trade and btic data
def merge_and_fill_es_btic(trade_df, btic_df):
    all_ts = pl.concat([
        trade_df.select("TIMESTAMP"),
        btic_df.select("TIMESTAMP")
    ]).unique().sort("TIMESTAMP")

    merged = all_ts.join_asof(trade_df.sort("TIMESTAMP"), on="TIMESTAMP", strategy="backward")
    merged = merged.join_asof(btic_df.sort("TIMESTAMP"), on="TIMESTAMP", strategy="backward")
    merged = merged.fill_null(strategy="forward").drop_nulls(["ES_TRADE", "BTIC"])

    return merged

In [4]:
# Helper function to merge the trade,btic merged datafram with theo_df to compute spreads
def merge_with_theo_and_compute_spreads(merged, theo_df):
    merged = merged.join_asof(theo_df.sort("TIMESTAMP"), on="TIMESTAMP", strategy="backward")
    merged = merged.drop_nulls(["REALTIME_INDEXPOINTS", "THEORETICAL_FUT"])

    merged = merged.with_columns([
        (pl.col("ES_TRADE") - pl.col("REALTIME_INDEXPOINTS")).alias("ES_MINUS_SPOT"),
        (pl.col("THEORETICAL_FUT") - pl.col("REALTIME_INDEXPOINTS")).alias("THEO_MINUS_SPOT")
    ]).drop_nulls(["ES_MINUS_SPOT", "THEO_MINUS_SPOT", "BTIC"])

    return merged

In [5]:
# Helper function for ploting the results
def plot_spreads(merged):
    ts = merged["TIMESTAMP"].to_numpy()
    es_spread = merged["ES_MINUS_SPOT"].to_numpy()
    theo_spread = merged["THEO_MINUS_SPOT"].to_numpy()
    btic = merged["BTIC"].to_numpy()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ts, y=es_spread, mode="lines", name="ES - Spot"))
    fig.add_trace(go.Scatter(x=ts, y=theo_spread, mode="lines", name="Theo - Spot"))
    fig.add_trace(go.Scatter(x=ts, y=btic, mode="lines", name="BTIC", line=dict(dash='dot')))

    fig.update_layout(
        title="Spread to Spot: ES vs Theoretical vs BTIC",
        xaxis_title="Timestamp",
        yaxis_title="Spread (Index Points)",
        legend_title="Series",
        template="plotly_white",
        hovermode="x unified"
    )

    fig.show()

In [7]:
# Example Usage
trade_url = "https://drive.google.com/uc?id=1snTABhzIgPTCx_Ugn0p7YuSSABlMsqrB"
btic_url = "https://drive.google.com/uc?id=1Gi43A0QDVAfrgfn0l87hyO-tsTjcJw_w"

# Load real data
trade_df, btic_df = load_trade_and_btic_data(trade_url, btic_url)

# Load theo_df over observed time range (change this with actual theo_df once we have it)
# This part is for genrating a random theo_df for now for visualization. 
# Delete this and replace with actual theo_df once we have it!!!!
def generate_theo_df(start_time, end_time):
    timestamps = []
    current_time = start_time
    while current_time <= end_time:
        timestamps.append(current_time)
        current_time += timedelta(minutes=1)

    n = len(timestamps)
    np.random.seed(42)

    index_base = 5800
    index_noise = np.cumsum(np.random.normal(loc=-0.01, scale=0.5, size=n))
    index_levels = index_base + index_noise
    div_points = np.random.uniform(0.5, 1.5, n)
    rates = np.random.uniform(0.04, 0.05, n)
    ttm = np.linspace(30, 26, n) / 365.0

    theo_df = pl.DataFrame({
        "TIMESTAMP": timestamps,
        "REALTIME_INDEXPOINTS": index_levels,
        "EXPECTED_DIVPOINTS": div_points,
        "INTEREST_RATES": rates,
        "TTM": ttm
    }).with_columns([
        (pl.col("REALTIME_INDEXPOINTS") * np.exp(pl.col("INTEREST_RATES") * pl.col("TTM"))
         - pl.col("EXPECTED_DIVPOINTS")).alias("THEORETICAL_FUT"),
        pl.col("TIMESTAMP").cast(pl.Datetime)
    ])

    return theo_df

min_time = min(trade_df["TIMESTAMP"].min(), btic_df["TIMESTAMP"].min())
max_time = max(trade_df["TIMESTAMP"].max(), btic_df["TIMESTAMP"].max())
theo_df = generate_theo_df(min_time, max_time)

# Forward-fill ES and BTIC
merged = merge_and_fill_es_btic(trade_df, btic_df)

# Merge with theo and compute spread
merged = merge_with_theo_and_compute_spreads(merged, theo_df)

# Plot the results
plot_spreads(merged)

Downloading...
From: https://drive.google.com/uc?id=1snTABhzIgPTCx_Ugn0p7YuSSABlMsqrB
To: /Users/HAOSHUWANG/trade_data.csv
100%|████████████████████████████████████████| 220k/220k [00:00<00:00, 9.49MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Gi43A0QDVAfrgfn0l87hyO-tsTjcJw_w
To: /Users/HAOSHUWANG/btic_data.csv
100%|██████████████████████████████████████| 46.3k/46.3k [00:00<00:00, 13.8MB/s]
